In [4]:
import pandas as pd
import numpy as np
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import sqlite3 as sq

In [5]:
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
GNRC = geodict.GNRC
GNRCFULLFIPS = geodict.GNRCFULLFIPS
namestocommon = geodict.namestocommon
KY = geodict.KY

In [6]:
def percent(x, y):
    return (x/y)*100

# Selecting LEHD datasets: 

https://lehd.ces.census.gov/data/lodes/LODES7/LODESTechDoc7.5.pdf  

Pages 5 & 6 of this document outlines the file structure for Origin-Destination data. This data is limited to disaggregations by the following categories:  
+ age (29 and younger, 30 to 54, and 55 or older)  
+ earnings (1,250 a month or less, 1,251 to 3,333 a month, and greater than 3,333 a month)  
+ Goods-Producing versus Trade, Transportation & Utilities, versus All Other Services industry sectors  

The earnings breakout is kind of useless so we'll only use the age and industry sector data.  

Page 7 of this document outlines the file structure for the Residence Area Characteristics data. This shows the same totals for age and earnings listed above, but further disaggregates the industry component into 2-digit NAICS codes. It also disaggregates job counts by race, Hispanic or Latino ethnicity, sex, and educational attainment (High School or Equivalent, Some college or Associate degree, Bachelor's degree or advanced degree). 

Page 8 of this document outlines the file structure for the Workplace Area Characteristics. This shows all of the same information as the RAC but also firm age and size. 

This data package also has a geography crosswalk which will be helpful in aggregating all of these numbers to a place and county level.


## Geography Crosswalk  
What we need from the geography crosswalk:  
+ a list of block groups in the GNRC region to narrow down the WAC and RAC preliminarily (we'll only be looking at the region here, not the state, because we're simply looking at in vs out instead of flow)  
+ a way to tag county and place to WAC, RAC, and OD block codes so that we can aggregate everything available by county, place, and unincorporated areas - dictionaries for now

In [7]:
url = 'https://lehd.ces.census.gov/data/lodes/LODES7/tn/tn_xwalk.csv.gz'
geo = pd.read_csv(url, dtype = str)
print('Your file is ready.')

Your file is ready.


In [8]:
#rename and drop unneeded columns
geocols = {'tabblk2010': 'Block Code', 'st': 'StateFIPS', 'stusps': 'StateUSPS', 'stname': 'State', 'cty': 'CountyFIPS', 'ctyname': 'County', 
           'trct': 'TractCODE', 'trctname': 'Tract', 'bgrp': 'BlockgroupCODE', 'bgrpname': 'Blockgroup', 'cbsa': 'CBSACODE', 'cbsaname': 'CBSA', 
           'zcta': 'ZCTACODE', 'zctaname': 'ZCTA', 'stplc': 'PlaceCODE', 'stplcname': 'Place', 'stwib': 'WIBCODE', 'stwibname': 'WIB', 
           'blklatdd': 'Lat', 'blklondd': 'Lon'}
geo.rename(columns = geocols, inplace = True)
cols = ['ctycsub', 'ctycsubname', 'stcd116', 'stcd116name', 'stsldl', 'stsldlname', 'stsldu', 'stslduname', 'stschool',
       'stschoolname', 'stsecon', 'stseconname', 'trib', 'tribname', 'tsub', 'tsubname', 'stanrc', 'stanrcname', 'necta', 'nectaname', 'mil',
       'milname']
geo = geo.drop(columns = cols)

In [9]:
geo.head()

,Block Code,StateFIPS,StateUSPS,State,CountyFIPS,County,TractCODE,Tract,BlockgroupCODE,Blockgroup,CBSACODE,CBSA,ZCTACODE,ZCTA,PlaceCODE,Place,WIBCODE,WIB,Lat,Lon,createdate
0,470279550001069,47,TN,Tennessee,47027,"Clay County, TN",47027955000,"9550 (Clay, TN)",470279550001,"1 (Tract 9550, Clay, TN)",99999,NaN,38551,38551,9999999,NaN,47LWIA07,07 Local Workforce Investment Area,36.5522497,-85.4519031,20211018
1,470059630002096,47,TN,Tennessee,47005,"Benton County, TN",47005963000,"9630 (Benton, TN)",470059630002,"2 (Tract 9630, Benton, TN)",99999,NaN,38221,38221,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,36.2107573,-88.1373529,20211018
2,470059630002091,47,TN,Tennessee,47005,"Benton County, TN",47005963000,"9630 (Benton, TN)",470059630002,"2 (Tract 9630, Benton, TN)",99999,NaN,38221,38221,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,36.1901840,-88.0977486,20211018
3,470059631001043,47,TN,Tennessee,47005,"Benton County, TN",47005963100,"9631 (Benton, TN)",470059631001,"1 (Tract 9631, Benton, TN)",99999,NaN,38320,38320,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,36.0788357,-88.0445962,20211018
4,470059630001061,47,TN,Tennessee,47005,"Benton County, TN",47005963000,"9630 (Benton, TN)",470059630001,"1 (Tract 9630, Benton, TN)",99999,NaN,38221,38221,9999999,NaN,47LWIA11,11 Local Workforce Investment Area,36.3057522,-88.0285893,20211018


In [10]:
#make a list of block codes in the GNRC region by listing county FIPS using geodict.py, indexing to only block codes in that list, and the creating a list of those
bg = geo.loc[geo['CountyFIPS'].isin(GNRCFULLFIPS)]
blockcodegnrcregion = list(bg['Block Code'].values)
#blockcodegnrcregion

In [11]:
#create a dictionary of county names to a list of their block codes
tncounties = list(geo['County'].unique())
c = {}
for i in tncounties:
    c['{}'.format(i)] = list(geo.loc[geo['County'] == '{}'.format(i), 'Block Code'])
#c

In [12]:
#invert that dictionary so that we can map it later
c_inverted = {}
for k,v in c.items():
    for x in v:
        c_inverted.setdefault(x,[]).append(k)
#c_inverted

In [13]:
#create a dictionary of place names to a list of their block codes
tnplace = geo[~geo['Place'].str.contains('CDP', na = True)]
tnplace = list(tnplace['Place'].unique())
p = {}
for i in tnplace:
    p['{}'.format(i)] = list(geo.loc[geo['Place'] == '{}'.format(i), 'Block Code'])
#p

In [14]:
#invert that dictionary so that we can map it later
p_inverted = {}
for k,v in p.items():
    for x in v:
        p_inverted.setdefault(x,[]).append(k)
#p_inverted

In [15]:
#create a tag for unincorporated block codes to use later per county
geo['Place'] = geo['Place'].fillna('None')
tnunincorporated = geo[(geo['Place'].str.contains('CDP'))|(geo['Place'] == 'None')]
test = list(tnunincorporated['Place'].unique())
un = {}
for i in test:
    un['{}'.format(i)] = list(geo.loc[geo['Place'] == '{}'.format(i), 'Block Code'])
#un

In [16]:
#invert that dictionary so that we can map it later
un_inverted = {}
for k,v in un.items():
    for x in v:
        un_inverted.setdefault(x,[]).append(k)
#un_inverted

## Residence Area Characteristics

In [17]:
#all primary jobs
url = 'https://lehd.ces.census.gov/data/lodes/LODES7/tn/rac/tn_rac_S000_JT01_2019.csv.gz'
rac = pd.read_csv(url)
print('Your file is ready.')

Your file is ready.


In [18]:
raccols = {'h_geocode': 'GEO_ID', 'C000': 'Total Jobs', 'CA01': 'Age:Workers 29 or Younger', 'CA02': 'Age:Workers 30 to 54', 
           'CA03': 'Age:Workers 55 Older', 'CE01': 'Earnings:1250mo or Less', 'CE02': 'Earnings:1251 to 3333mo', 
           'CE03': 'Earnings:Greater Than 3333mo', 'CNS01': 'NAICS:11Agriculture, Forestry, Fishing & Hunting', 
           'CNS02': 'NAICS:21Mining, Quarrying, Oil & Gas', 'CNS03': 'NAICS:22Utilities', 'CNS04': 'NAICS:23Construction', 'CNS05':'NAICS:31to33Manufacturing',
           'CNS06': 'NAICS:42Wholesale Trade', 'CNS07': 'NAICS:44to45Retail Trade', 'CNS08': 'NAICS:48to49Transportation & Warehousing',
           'CNS09': 'NAICS:51Information', 'CNS10': 'NAICS:52Finance & Insurance', 'CNS11': 'NAICS:53Real Estate & Rental & Leasing', 
           'CNS12': 'NAICS:54Professional, Scientific, & Technical Services', 'CNS13': 'NAICS:55Management of Companies & Enterprises', 
           'CNS14': 'NAICS:56Administrative & Support & Waste Management & Remediation Services', 'CNS15': 'NAICS:61Educational Services', 
           'CNS16': 'NAICS:62Healthcare & Social Assistance', 'CNS17': 'NAICS:71Arts, Entertainment, & Recreation', 'CNS18': 'NAICS:72Accommodation & Food Services', 
           'CNS19': 'NAICS:81Other Services (Except Public Administration)', 'CNS20': 'NAICS:92Public Administration', 
           'CR01': 'Race:White Alone', 'CR02': 'Race:Black or African American Alone', 'CR03': 'Race:American Indian or Alaska Native Alone', 
           'CR04': 'Race:Asian Alone', 'CR05': 'Race:Native Hawaiian or Other Pacific Islander Alone', 'CR07': 'Race:Two or More Races', 
           'CT01': 'Ethnicity:Not Hispanic or Latino', 'CT02': 'Ethnicity:Hispanic or Latino', 'CD01':'Ed:Less than High School', 
           'CD02': 'Ed:High School or Equivalent', 'CD03': 'Ed:Some College or Associates', 'CD04': 'Ed:Bachelors or Advanced Degree', 'CS01': 'Sex:Male', 
           'CS02': 'Sex:Female'}

In [19]:
rac.rename(columns = raccols, inplace = True)

In [20]:
rac = rac.set_index('GEO_ID')
rac = rac.add_prefix('RAC:')

In [21]:
rac.head()

,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation & Warehousing,RAC:NAICS:51Information,RAC:NAICS:52Finance & Insurance,RAC:NAICS:53Real Estate & Rental & Leasing,"RAC:NAICS:54Professional, Scientific, & Technical Services",RAC:NAICS:55Management of Companies & Enterprises,RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS:61Educational Services,RAC:NAICS:62Healthcare & Social Assistance,"RAC:NAICS:71Arts, Entertainment, & Recreation",RAC:NAICS:72Accommodation & Food Services,RAC:NAICS:81Other Services (Except Public Administration),RAC:NAICS:92Public Administration,RAC:Race:White Alone,RAC:Race:Black or African American Alone,RAC:Race:American Indian or Alaska Native Alone,RAC:Race:Asian Alone,RAC:Race:Native Hawaiian or Other Pacific Islander Alone,RAC:Race:Two or More Races,RAC:Ethnicity:Not Hispanic or Latino,RAC:Ethnicity:Hispanic or Latino,RAC:Ed:Less than High School,RAC:Ed:High School or Equivalent,RAC:Ed:Some College or Associates,RAC:Ed:Bachelors or Advanced Degree,RAC:Sex:Male,RAC:Sex:Female,RAC:createdate
GEO_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
470010201001006,76,28,39,9,15,31,30,0,0,0,3,6,2,6,1,2,5,1,9,0,9,7,12,0,9,3,1,66,7,2,1,0,0,66,10,8,17,16,7,40,36,20211018
470010201001015,125,46,61,18,26,49,50,0,0,0,1,15,4,21,1,1,4,4,13,3,11,8,14,1,18,4,2,91,15,0,14,0,5,117,8,6,27,26,20,66,59,20211018
470010201001057,8,2,5,1,1,4,3,0,0,0,0,1,0,2,1,0,0,0,1,0,1,0,0,0,0,0,2,7,1,0,0,0,0,8,0,0,2,2,2,5,3,20211018
470010201001062,95,32,41,22,18,39,38,0,0,0,3,10,5,14,1,1,3,2,11,4,11,5,10,1,11,3,0,82,6,0,5,0,2,92,3,3,17,16,27,49,46,20211018
470010201001063,80,19,44,17,16,32,32,0,0,0,1,9,1,11,2,0,4,2,6,3,5,7,8,1,14,4,2,61,8,0,8,0,3,78,2,12,13,20,16,35,45,20211018


## Workplace Area Characteristics

In [22]:
#all primary jobs
url = 'https://lehd.ces.census.gov/data/lodes/LODES7/tn/wac/tn_wac_S000_JT01_2019.csv.gz'
wac = pd.read_csv(url)
print('Your file is ready.')

Your file is ready.


In [23]:
waccols = {'w_geocode': 'GEO_ID', 'C000': 'Total Jobs', 'CA01': 'Age:Workers 29 or Younger', 'CA02': 'Age:Workers 30 to 54', 
           'CA03': 'Age:Workers 55 Older', 'CE01': 'Earnings:1250mo or Less', 'CE02': 'Earnings:1251 to 3333mo', 
           'CE03': 'Earnings:Greater Than 3333mo', 'CNS01': 'NAICS:11Agriculture, Forestry, Fishing & Hunting', 
           'CNS02': 'NAICS:21Mining, Quarrying, Oil & Gas', 'CNS03': 'NAICS:22Utilities', 'CNS04': 'NAICS:23Construction', 'CNS05':'NAICS:31to33Manufacturing',
           'CNS06': 'NAICS:42Wholesale Trade', 'CNS07': 'NAICS:44to45Retail Trade', 'CNS08': 'NAICS:48to49Transportation & Warehousing',
           'CNS09': 'NAICS:51Information', 'CNS10': 'NAICS:52Finance & Insurance', 'CNS11': 'NAICS:53Real Estate & Rental & Leasing', 
           'CNS12': 'NAICS:54Professional, Scientific, & Technical Services', 'CNS13': 'NAICS:55Management of Companies & Enterprises', 
           'CNS14': 'NAICS:56Administrative & Support & Waste Management & Remediation Services', 'CNS15': 'NAICS:61Educational Services', 
           'CNS16': 'NAICS:62Healthcare & Social Assistance', 'CNS17': 'NAICS:71Arts, Entertainment, & Recreation', 'CNS18': 'NAICS:72Accommodation & Food Services', 
           'CNS19': 'NAICS:81Other Services (Except Public Administration)', 'CNS20': 'NAICS:92Public Administration', 
           'CR01': 'Race:White Alone', 'CR02': 'Race:Black or African American Alone', 'CR03': 'Race:American Indian or Alaska Native Alone', 
           'CR04': 'Race:Asian Alone', 'CR05': 'Race:Native Hawaiian or Other Pacific Islander Alone', 'CR07': 'Race:Two or More Races', 
           'CT01': 'Ethnicity:Not Hispanic or Latino', 'CT02': 'Ethnicity:Hispanic or Latino', 'CD01':'Ed:Less than High School', 
           'CD02': 'Ed:High School or Equivalent', 'CD03': 'Ed:Some College or Associates', 'CD04': 'Ed:Bachelors or Advanced Degree', 'CS01': 'Sex:Male', 
           'CS02': 'Sex:Female', 'CFA01': 'FirmAge:0 to 1 Year', 'CFA02': 'FirmAge:2 to 3 Years', 'CFA03': 'FirmAge:4 to 5 Years', 'CFA04': 'FirmAge:6 to 10 Years',
           'CFA05': 'FirmAge:11 or More Years', 'CFS01': 'FirmSize:0 to 19 Employees', 'CFS02': 'FirmSize:20 to 49 Employees', 
           'CFS03': 'FirmSize:50 to 249 Employees', 'CFS04': 'FirmSize:250 to 499 Employees', 'CFS05': 'FirmSize:500 or More Employees'}

In [24]:
wac.rename(columns = waccols, inplace = True)

In [25]:
wac = wac.set_index('GEO_ID')
wac = wac.add_prefix('WAC:')

In [26]:
wac.head()

,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,WAC:createdate
GEO_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
470010201001000,97,35,42,20,30,48,19,0,0,0,1,0,0,11,0,0,21,0,1,0,0,0,16,11,16,20,0,91,3,0,1,0,2,93,4,7,19,23,13,29,68,0,0,0,0,0,0,0,0,0,0,20211018
470010201001005,38,7,14,17,9,23,6,0,0,0,0,0,0,5,0,0,2,0,6,0,0,0,0,0,25,0,0,30,2,1,4,0,1,38,0,2,10,12,7,8,30,0,0,0,0,0,0,0,0,0,0,20211018
470010201001006,3,0,1,2,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,3,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,20211018
470010201001009,176,39,93,44,6,94,76,0,0,0,0,0,0,0,0,0,166,0,0,0,0,0,0,0,10,0,0,161,7,0,5,0,3,169,7,12,43,44,38,44,132,0,0,0,0,0,0,0,0,0,0,20211018
470010201001010,4,1,3,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,4,0,0,4,0,1,0,1,1,2,2,0,0,0,0,0,0,0,0,0,0,20211018


### Join RAC and WAC

In [27]:
#reset index to join on GEO_ID and filter both to only include the GNRC region
rac = rac.reset_index(drop = False)
rac['GEO_ID'] = rac['GEO_ID'].astype(str)
rac = rac.loc[rac['GEO_ID'].isin(blockcodegnrcregion)]
wac = wac.reset_index(drop = False)
wac['GEO_ID'] = wac['GEO_ID'].astype(str)
wac = wac.loc[wac['GEO_ID'].isin(blockcodegnrcregion)]

In [28]:
df = pd.merge(left = wac, right = rac, on = 'GEO_ID', how = 'outer')

In [29]:
df.head()

,GEO_ID,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,WAC:createdate,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation & Warehousing,RAC:NAICS:51Information,RAC:NAICS:52Finance & Insurance,RAC:NAICS:53Real Estate & Rental & Leasing,"RAC:NAICS:54Professional, Scientific, & Technical Services",RAC:NAICS:55Management of Companies & Enterprises,RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS:61Educational Services,RAC:NAICS:62Healthcare & Social Assistance,"RAC:NAICS:71Arts, Entertainment, & Recreation",RAC:NAICS:72Accommodation & Food Services,RAC:NAICS:81Other Services (Except Public Administration),RAC:NAICS:92Public Administration,RAC:Race:White Alone,RAC:Race:Black or African American Alone,RAC:Race:American Indian or Alaska Native Alone,RAC:Race:Asian Alone,RAC:Race:Native Hawaiian or Other Pacific Islander Alone,RAC:Race:Two or More Races,RAC:Ethnicity:Not Hispanic or Latino,RAC:Ethnicity:Hispanic or Latino,RAC:Ed:Less than High School,RAC:Ed:High School or Equivalent,RAC:Ed:Some College or Associates,RAC:Ed:Bachelors or Advanced Degree,RAC:Sex:Male,RAC:Sex:Female,RAC:createdate
0,470210701021001,7.0,1.0,2.0,4.0,1.0,4.0,2.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,1.0,4.0,0.0,1.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20211018.0,34.0,11.0,15.0,8.0,3.0,17.0,14.0,0.0,0.0,1.0,1.0,6.0,0.0,7.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,2.0,7.0,1.0,3.0,0.0,1.0,32.0,1.0,0.0,0.0,1.0,0.0,34.0,0.0,7.0,4.0,7.0,5.0,20.0,14.0,20211018.0
1,470210701021002,67.0,12.0,38.0,17.0,8.0,33.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0,0.0,0.0,0.0,5.0,57.0,6.0,3.0,0.0,0.0,1.0,64.0,3.0,5.0,18.0,16.0,16.0,11.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20211018.0,89.0,23.0,41.0,25.0,18.0,26.0,45.0,0.0,0.0,0.0,10.0,10.0,2.0,10.0,3.0,3.0,4.0,1.0,3.0,2.0,8.0,4.0,

In [30]:
#make dataframes to be grouped by both county and place to concatenate later
county = df
place = df

In [31]:
#add columns with county name and place name respectively
county['County'] = county['GEO_ID'].map(c_inverted)
county['Incorporated'] = county['GEO_ID'].map(un_inverted)
place['Place'] = place['GEO_ID'].map(p_inverted)

In [32]:
county['County'] = county['County'].str.get(0)
county['Incorporated'] = county['Incorporated'].str.get(0)
place['Place'] = place['Place'].str.get(0)

In [33]:
county = county.drop(columns = ['GEO_ID'])
place = place.drop(columns = ['GEO_ID'])

In [34]:
countywacrac = county.groupby('County').sum()

C:\Users\jmccall\AppData\Local\Temp\ipykernel_13780\1007141463.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  countywacrac = county.groupby('County').sum()


In [35]:
placewacrac = place.groupby('Place').sum()

C:\Users\jmccall\AppData\Local\Temp\ipykernel_13780\2637243798.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  placewacrac = place.groupby('Place').sum()


In [36]:
unwacrac = county.groupby(['Incorporated', 'County']).sum()

C:\Users\jmccall\AppData\Local\Temp\ipykernel_13780\3120419885.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  unwacrac = county.groupby(['Incorporated', 'County']).sum()


In [37]:
unwacrac = unwacrac.reset_index(drop = False)
unwacrac = unwacrac.loc[unwacrac['Incorporated'] == 'None']

In [38]:
unwacrac = unwacrac.drop(columns = 'Incorporated')
unwacrac = unwacrac.set_index('County').transpose()

In [39]:
namesdict = {'Cheatham County, TN': 'Cheatham Unincorporated', 
             'Dickson County, TN': 'Dickson Unincorporated',
             'Houston County, TN': 'Houston Unincorporated',
             'Humphreys County, TN': 'Humphreys Unincorporated',
             'Maury County, TN': 'Maury Unincorporated',
             'Montgomery County, TN': 'Montgomery Unincorporated',
             'Robertson County, TN': 'Robertson Unincorporated',
             'Rutherford County, TN': 'Rutherford Unincorporated',
             'Stewart County, TN': 'Stewart Unincorporated',
             'Sumner County, TN': 'Sumner Unincorporated',
             'Williamson County, TN': 'Williamson Unincorporated',
             'Wilson County, TN': 'Wilson Unincorporated'}
unwacrac = unwacrac.rename(columns = namesdict)

In [40]:
unwacrac = unwacrac.transpose()

In [41]:
unwacrac = unwacrac.reset_index(drop = False)
unwacrac = unwacrac.rename(columns = {'County': 'NAME'})

In [42]:
unwacrac.set_index('NAME', inplace = True)

In [43]:
unwacrac.head()

,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,WAC:createdate,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation & Warehousing,RAC:NAICS:51Information,RAC:NAICS:52Finance & Insurance,RAC:NAICS:53Real Estate & Rental & Leasing,"RAC:NAICS:54Professional, Scientific, & Technical Services",RAC:NAICS:55Management of Companies & Enterprises,RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS:61Educational Services,RAC:NAICS:62Healthcare & Social Assistance,"RAC:NAICS:71Arts, Entertainment, & Recreation",RAC:NAICS:72Accommodation & Food Services,RAC:NAICS:81Other Services (Except Public Administration),RAC:NAICS:92Public Administration,RAC:Race:White Alone,RAC:Race:Black or African American Alone,RAC:Race:American Indian or Alaska Native Alone,RAC:Race:Asian Alone,RAC:Race:Native Hawaiian or Other Pacific Islander Alone,RAC:Race:Two or More Races,RAC:Ethnicity:Not Hispanic or Latino,RAC:Ethnicity:Hispanic or Latino,RAC:Ed:Less than High School,RAC:Ed:High School or Equivalent,RAC:Ed:Some College or Associates,RAC:Ed:Bachelors or Advanced Degree,RAC:Sex:Male,RAC:Sex:Female,RAC:createdate
NAME,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cheatham Unincorporated,1532.0,405.0,796.0,331.0,328.0,641.0,563.0,4.0,0.0,0.0,406.0,203.0,22.0,145.0,78.0,10.0,14.0,22.0,36.0,0.0,151.0,1.0,201.0,13.0,108.0,113.0,5.0,1413.0,80.0,10.0,12.0,2.0,15.0,1447.0,85.0,169.0,380.0,335.0,243.0,898.0,634.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.112497e+09,12870.0,2894.0,6898.0,3078.0,1950.0,4032.0,6888.0,17.0,6.0,85.0,1108.0,1301.0,595.0,1398.0,652.0,268.0,488.0,186.0,790.0,242.0,769.0,1005.0,1662.0,169.0,948.0,402.0,779.0,12231.0,350.0,40.0,99.0,10.0,140.0,12478.0,392.0,1039.0,2992.0,3180.0,2765.0,6657.0,6213.0,9.580023e+09
Dickson Unincorporated,3572.0,1027.0,1791.0,754.0,792.0,1429.0,1351.0,26.0,0.0,22.0,620.0,837.0

In [141]:
countywacrac.head()

,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,WAC:createdate,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation & Warehousing,RAC:NAICS:51Information,RAC:NAICS:52Finance & Insurance,RAC:NAICS:53Real Estate & Rental & Leasing,"RAC:NAICS:54Professional, Scientific, & Technical Services",RAC:NAICS:55Management of Companies & Enterprises,RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS:61Educational Services,RAC:NAICS:62Healthcare & Social Assistance,"RAC:NAICS:71Arts, Entertainment, & Recreation",RAC:NAICS:72Accommodation & Food Services,RAC:NAICS:81Other Services (Except Public Administration),RAC:NAICS:92Public Administration,RAC:Race:White Alone,RAC:Race:Black or African American Alone,RAC:Race:American Indian or Alaska Native Alone,RAC:Race:Asian Alone,RAC:Race:Native Hawaiian or Other Pacific Islander Alone,RAC:Race:Two or More Races,RAC:Ethnicity:Not Hispanic or Latino,RAC:Ethnicity:Hispanic or Latino,RAC:Ed:Less than High School,RAC:Ed:High School or Equivalent,RAC:Ed:Some College or Associates,RAC:Ed:Bachelors or Advanced Degree,RAC:Sex:Male,RAC:Sex:Female,RAC:createdate
County,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Cheatham County, TN",7918.0,1949.0,4255.0,1714.0,1433.0,2789.0,3696.0,4.0,0.0,44.0,784.0,2290.0,76.0,820.0,382.0,20.0,165.0,69.0,263.0,2.0,393.0,960.0,447.0,62.0,600.0,172.0,365.0,7053.0,638.0,30.0,103.0,13.0,81.0,7482.0,436.0,784.0,1981.0,1833.0,1371.0,4295.0,3623.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.396342e+09,19706.0,4446.0,10549.0,4711.0,3016.0,6117.0,10573.0,25.0,11.0,133.0,1584.0,1888.0,951.0,2087.0,999.0,405.0,781.0,312.0,1245.0,388.0,1186.0,1545.0,2556.0,269.0,1518.0,605.0,1218.0,18645.0,599.0,67.0,162.0,11.0,222.0,19050.0,656.0,1601.0,4477.0,4908.0,4274.0,10195.0,9511.0,1.402645e+10
"Davidson County, TN",502956.0,122426.0,272971.0,107559.0,758

In [144]:
placewacrac.head()

,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,WAC:createdate,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation & Warehousing,RAC:NAICS:51Information,RAC:NAICS:52Finance & Insurance,RAC:NAICS:53Real Estate & Rental & Leasing,"RAC:NAICS:54Professional, Scientific, & Technical Services",RAC:NAICS:55Management of Companies & Enterprises,RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS:61Educational Services,RAC:NAICS:62Healthcare & Social Assistance,"RAC:NAICS:71Arts, Entertainment, & Recreation",RAC:NAICS:72Accommodation & Food Services,RAC:NAICS:81Other Services (Except Public Administration),RAC:NAICS:92Public Administration,RAC:Race:White Alone,RAC:Race:Black or African American Alone,RAC:Race:American Indian or Alaska Native Alone,RAC:Race:Asian Alone,RAC:Race:Native Hawaiian or Other Pacific Islander Alone,RAC:Race:Two or More Races,RAC:Ethnicity:Not Hispanic or Latino,RAC:Ethnicity:Hispanic or Latino,RAC:Ed:Less than High School,RAC:Ed:High School or Equivalent,RAC:Ed:Some College or Associates,RAC:Ed:Bachelors or Advanced Degree,RAC:Sex:Male,RAC:Sex:Female,RAC:createdate
Place,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Adams city, TN",46.0,13.0,25.0,8.0,28.0,13.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,12.0,0.0,9.0,0.0,2.0,41.0,2.0,1.0,1.0,0.0,1.0,45.0,1.0,3.0,14.0,13.0,3.0,10.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.010551e+08,321.0,71.0,169.0,81.0,55.0,104.0,162.0,3.0,0.0,7.0,23.0,37.0,15.0,35.0,15.0,4.0,8.0,6.0,11.0,9.0,21.0,20.0,41.0,3.0,25.0,11.0,27.0,291.0,17.0,8.0,3.0,1.0,1.0,309.0,12.0,27.0,78.0,88.0,57.0,161.0,160.0,6.265416e+08
"Ashland City town, TN",4691.0,1001.0,2611.0,1079.0,651.0,1536.0,2504.0,0.0,0.0,15.0,121.0,1920.0,1.0,495.0,258.0,8.0,74.0,21.0,69.0,0.0,129.0,873.0,99.0,5.0,269.0,18.0,316.0,4107.0,449.0,16.0,70.0,3.0,46.0,4

In [145]:
wacrac = countywacrac.append(placewacrac)
#wacrac = wacrac.rename(columns = tnonlytocommon)
data = wacrac.append(unwacrac)

C:\Users\jmccall\AppData\Local\Temp\ipykernel_10136\769542372.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wacrac = countywacrac.append(placewacrac)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_10136\769542372.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = wacrac.append(unwacrac)


In [146]:
data = data.transpose().rename(columns = namestocommon)
data = data.transpose().reset_index(drop = False)

In [147]:
data.rename(columns = {'index': 'NAME'}, inplace = True)

In [148]:
data.head()

,NAME,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,WAC:createdate,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation & Warehousing,RAC:NAICS:51Information,RAC:NAICS:52Finance & Insurance,RAC:NAICS:53Real Estate & Rental & Leasing,"RAC:NAICS:54Professional, Scientific, & Technical Services",RAC:NAICS:55Management of Companies & Enterprises,RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS:61Educational Services,RAC:NAICS:62Healthcare & Social Assistance,"RAC:NAICS:71Arts, Entertainment, & Recreation",RAC:NAICS:72Accommodation & Food Services,RAC:NAICS:81Other Services (Except Public Administration),RAC:NAICS:92Public Administration,RAC:Race:White Alone,RAC:Race:Black or African American Alone,RAC:Race:American Indian or Alaska Native Alone,RAC:Race:Asian Alone,RAC:Race:Native Hawaiian or Other Pacific Islander Alone,RAC:Race:Two or More Races,RAC:Ethnicity:Not Hispanic or Latino,RAC:Ethnicity:Hispanic or Latino,RAC:Ed:Less than High School,RAC:Ed:High School or Equivalent,RAC:Ed:Some College or Associates,RAC:Ed:Bachelors or Advanced Degree,RAC:Sex:Male,RAC:Sex:Female,RAC:createdate
0,Cheatham County,7918.0,1949.0,4255.0,1714.0,1433.0,2789.0,3696.0,4.0,0.0,44.0,784.0,2290.0,76.0,820.0,382.0,20.0,165.0,69.0,263.0,2.0,393.0,960.0,447.0,62.0,600.0,172.0,365.0,7053.0,638.0,30.0,103.0,13.0,81.0,7482.0,436.0,784.0,1981.0,1833.0,1371.0,4295.0,3623.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.396342e+09,19706.0,4446.0,10549.0,4711.0,3016.0,6117.0,10573.0,25.0,11.0,133.0,1584.0,1888.0,951.0,2087.0,999.0,405.0,781.0,312.0,1245.0,388.0,1186.0,1545.0,2556.0,269.0,1518.0,605.0,1218.0,18645.0,599.0,67.0,162.0,11.0,222.0,19050.0,656.0,1601.0,4477.0,4908.0,4274.0,10195.0,9511.0,1.402645e+10
1,Davidson County,502956.0,122426.0,272971.0,107559.0,75893.0,147811.0,279252.0,261.0,182.0,1465.0,23603.0,21374.0,24395.0,42556.0,26896.0,14044.0,23276.0,10106.

In [149]:
wacrac = data.set_index('NAME')

In [150]:
wacrac = wacrac.drop(columns = ['RAC:createdate', 'WAC:createdate'])

In [151]:
wacrac.head()

,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation & Warehousing,RAC:NAICS:51Information,RAC:NAICS:52Finance & Insurance,RAC:NAICS:53Real Estate & Rental & Leasing,"RAC:NAICS:54Professional, Scientific, & Technical Services",RAC:NAICS:55Management of Companies & Enterprises,RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS:61Educational Services,RAC:NAICS:62Healthcare & Social Assistance,"RAC:NAICS:71Arts, Entertainment, & Recreation",RAC:NAICS:72Accommodation & Food Services,RAC:NAICS:81Other Services (Except Public Administration),RAC:NAICS:92Public Administration,RAC:Race:White Alone,RAC:Race:Black or African American Alone,RAC:Race:American Indian or Alaska Native Alone,RAC:Race:Asian Alone,RAC:Race:Native Hawaiian or Other Pacific Islander Alone,RAC:Race:Two or More Races,RAC:Ethnicity:Not Hispanic or Latino,RAC:Ethnicity:Hispanic or Latino,RAC:Ed:Less than High School,RAC:Ed:High School or Equivalent,RAC:Ed:Some College or Associates,RAC:Ed:Bachelors or Advanced Degree,RAC:Sex:Male,RAC:Sex:Female
NAME,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Cheatham County,7918.0,1949.0,4255.0,1714.0,1433.0,2789.0,3696.0,4.0,0.0,44.0,784.0,2290.0,76.0,820.0,382.0,20.0,165.0,69.0,263.0,2.0,393.0,960.0,447.0,62.0,600.0,172.0,365.0,7053.0,638.0,30.0,103.0,13.0,81.0,7482.0,436.0,784.0,1981.0,1833.0,1371.0,4295.0,3623.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19706.0,4446.0,10549.0,4711.0,3016.0,6117.0,10573.0,25.0,11.0,133.0,1584.0,1888.0,951.0,2087.0,999.0,405.0,781.0,312.0,1245.0,388.0,1186.0,1545.0,2556.0,269.0,1518.0,605.0,1218.0,18645.0,599.0,67.0,162.0,11.0,222.0,19050.0,656.0,1601.0,4477.0,4908.0,4274.0,10195.0,9511.0
Davidson County,502956.0,122426.0,272971.0,107559.0,75893.0,147811.0,279252.0,261.0,182.0,1465.0,23603.0,21374.0,24395.0,42556.

In [152]:
wacrac['WAC:Age%:Workers 29 or Younger'] = percent(wacrac['WAC:Age:Workers 29 or Younger'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Age%:Workers 30 to 54'] = percent(wacrac['WAC:Age:Workers 30 to 54'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Age%:Workers 55 Older'] = percent(wacrac['WAC:Age:Workers 55 Older'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Earnings%:1250mo or Less'] = percent(wacrac['WAC:Earnings:1250mo or Less'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Earnings%:1251 to 3333mo'] = percent(wacrac['WAC:Earnings:1251 to 3333mo'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Earnings%:Greater Than 3333mo'] = percent(wacrac['WAC:Earnings:Greater Than 3333mo'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:11Agriculture, Forestry, Fishing & Hunting'] = percent(wacrac['WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:21Mining, Quarrying, Oil & Gas'] = percent(wacrac['WAC:NAICS:21Mining, Quarrying, Oil & Gas'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:22Utilities'] = percent(wacrac['WAC:NAICS:22Utilities'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:23Construction'] = percent(wacrac['WAC:NAICS:23Construction'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:31to33Manufacturing'] = percent(wacrac['WAC:NAICS:31to33Manufacturing'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:42Wholesale Trade'] = percent(wacrac['WAC:NAICS:42Wholesale Trade'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:44to45Retail Trade'] = percent(wacrac['WAC:NAICS:44to45Retail Trade'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:48to49Transportation & Warehousing'] = percent(wacrac['WAC:NAICS:48to49Transportation & Warehousing'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:51Information'] = percent(wacrac['WAC:NAICS:51Information'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:52Finance & Insurance'] = percent(wacrac['WAC:NAICS:52Finance & Insurance'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:48to49Transportation & Warehousing'] = percent(wacrac['WAC:NAICS:48to49Transportation & Warehousing'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:51Information'] = percent(wacrac['WAC:NAICS:51Information'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:52Finance & Insurance'] = percent(wacrac['WAC:NAICS:52Finance & Insurance'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:53Real Estate & Rental & Leasing'] = percent(wacrac['WAC:NAICS:53Real Estate & Rental & Leasing'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:54Professional, Scientific, & Technical Services'] = percent(wacrac['WAC:NAICS:54Professional, Scientific, & Technical Services'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:55Management of Companies & Enterprises'] = percent(wacrac['WAC:NAICS:55Management of Companies & Enterprises'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:56Administrative & Support & Waste Management & Remediation Services'] = percent(wacrac['WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services'], 
                                                                                             wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:61Educational Services'] = percent(wacrac['WAC:NAICS:61Educational Services'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:62Healthcare & Social Assistance'] = percent(wacrac['WAC:NAICS:62Healthcare & Social Assistance'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:71Arts, Entertainment, & Recreation'] = percent(wacrac['WAC:NAICS:71Arts, Entertainment, & Recreation'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:72Accommodation & Food Services'] = percent(wacrac['WAC:NAICS:72Accommodation & Food Services'], wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:81Other Services (Except Public Administration)'] = percent(wacrac['WAC:NAICS:81Other Services (Except Public Administration)'], 
                                                                               wacrac['WAC:Total Jobs'])
wacrac['WAC:NAICS%:92Public Administration'] = percent(wacrac['WAC:NAICS:92Public Administration'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Race%:White Alone'] = percent(wacrac['WAC:Race:White Alone'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Race%:Black or African American Alone'] = percent(wacrac['WAC:Race:Black or African American Alone'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Race%:American Indian or Alaska Native Alone'] = percent(wacrac['WAC:Race:American Indian or Alaska Native Alone'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Race%:Asian Alone'] = percent(wacrac['WAC:Race:Asian Alone'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Race%:Native Hawaiian or Other Pacific Islander Alone'] = percent(wacrac['WAC:Race:Native Hawaiian or Other Pacific Islander Alone'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Race%:Two or More Races'] = percent(wacrac['WAC:Race:Two or More Races'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Ethnicity%:Not Hispanic or Latino'] = percent(wacrac['WAC:Ethnicity:Not Hispanic or Latino'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Ethnicity%:Hispanic or Latino'] = percent(wacrac['WAC:Ethnicity:Hispanic or Latino'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Ed%:Less than High School'] = percent(wacrac['WAC:Ed:Less than High School'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Ed%:High School or Equivalent'] = percent(wacrac['WAC:Ed:High School or Equivalent'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Ed%:Some College or Associates'] = percent(wacrac['WAC:Ed:Some College or Associates'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Ed%:Bachelors or Advanced Degree'] = percent(wacrac['WAC:Ed:Bachelors or Advanced Degree'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Sex%:Male'] = percent(wacrac['WAC:Sex:Male'], wacrac['WAC:Total Jobs'])
wacrac['WAC:Sex%:Female'] = percent(wacrac['WAC:Sex:Female'], wacrac['WAC:Total Jobs'])
wacrac['WAC:FirmAge%:0 to 1 Year'] = percent(wacrac['WAC:FirmAge:0 to 1 Year'], wacrac['WAC:Total Jobs'])
wacrac['WAC:FirmAge%:2 to 3 Years'] = percent(wacrac['WAC:FirmAge:2 to 3 Years'], wacrac['WAC:Total Jobs'])
wacrac['WAC:FirmAge%:4 to 5 Years'] = percent(wacrac['WAC:FirmAge:4 to 5 Years'], wacrac['WAC:Total Jobs'])
wacrac['WAC:FirmAge%:6 to 10 Years'] = percent(wacrac['WAC:FirmAge:6 to 10 Years'], wacrac['WAC:Total Jobs'])
wacrac['WAC:FirmAge%:11 or More Years'] = percent(wacrac['WAC:FirmAge:11 or More Years'], wacrac['WAC:Total Jobs'])
wacrac['WAC:FirmSize%:0 to 19 Employees'] = percent(wacrac['WAC:FirmSize:0 to 19 Employees'], wacrac['WAC:Total Jobs'])
wacrac['WAC:FirmSize%:20 to 49 Employees'] = percent(wacrac['WAC:FirmSize:20 to 49 Employees'], wacrac['WAC:Total Jobs'])
wacrac['WAC:FirmSize%:50 to 249 Employees'] = percent(wacrac['WAC:FirmSize:50 to 249 Employees'], wacrac['WAC:Total Jobs'])
wacrac['WAC:FirmSize%:250 to 499 Employees'] = percent(wacrac['WAC:FirmSize:250 to 499 Employees'], wacrac['WAC:Total Jobs'])
wacrac['WAC:FirmSize%:500 or More Employees'] = percent(wacrac['WAC:FirmSize:500 or More Employees'], wacrac['WAC:Total Jobs'])

In [153]:
wacrac['RAC:Age%:Workers 29 or Younger'] = percent(wacrac['RAC:Age:Workers 29 or Younger'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Age%:Workers 30 to 54'] = percent(wacrac['RAC:Age:Workers 30 to 54'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Age%:Workers 55 Older'] = percent(wacrac['RAC:Age:Workers 55 Older'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Earnings%:1250mo or Less'] = percent(wacrac['RAC:Earnings:1250mo or Less'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Earnings%:1251 to 3333mo'] = percent(wacrac['RAC:Earnings:1251 to 3333mo'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Earnings%:Greater Than 3333mo'] = percent(wacrac['RAC:Earnings:Greater Than 3333mo'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:11Agriculture, Forestry, Fishing & Hunting'] = percent(wacrac['RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:21Mining, Quarrying, Oil & Gas'] = percent(wacrac['RAC:NAICS:21Mining, Quarrying, Oil & Gas'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:22Utilities'] = percent(wacrac['RAC:NAICS:22Utilities'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:23Construction'] = percent(wacrac['RAC:NAICS:23Construction'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:31to33Manufacturing'] = percent(wacrac['RAC:NAICS:31to33Manufacturing'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:42Wholesale Trade'] = percent(wacrac['RAC:NAICS:42Wholesale Trade'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:44to45Retail Trade'] = percent(wacrac['RAC:NAICS:44to45Retail Trade'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:48to49Transportation & Warehousing'] = percent(wacrac['RAC:NAICS:48to49Transportation & Warehousing'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:51Information'] = percent(wacrac['RAC:NAICS:51Information'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:52Finance & Insurance'] = percent(wacrac['RAC:NAICS:52Finance & Insurance'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:48to49Transportation & Warehousing'] = percent(wacrac['RAC:NAICS:48to49Transportation & Warehousing'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:51Information'] = percent(wacrac['RAC:NAICS:51Information'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:52Finance & Insurance'] = percent(wacrac['RAC:NAICS:52Finance & Insurance'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:53Real Estate & Rental & Leasing'] = percent(wacrac['RAC:NAICS:53Real Estate & Rental & Leasing'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:54Professional, Scientific, & Technical Services'] = percent(wacrac['RAC:NAICS:54Professional, Scientific, & Technical Services'], 
                                                                                wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:55Management of Companies & Enterprises'] = percent(wacrac['RAC:NAICS:55Management of Companies & Enterprises'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:56Administrative & Support & Waste Management & Remediation Services'] = percent(wacrac['RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services'], 
                                                                                             wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:61Educational Services'] = percent(wacrac['RAC:NAICS:61Educational Services'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:62Healthcare & Social Assistance'] = percent(wacrac['RAC:NAICS:62Healthcare & Social Assistance'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:71Arts, Entertainment, & Recreation'] = percent(wacrac['RAC:NAICS:71Arts, Entertainment, & Recreation'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:72Accommodation & Food Services'] = percent(wacrac['RAC:NAICS:72Accommodation & Food Services'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:81Other Services (Except Public Administration)'] = percent(wacrac['RAC:NAICS:81Other Services (Except Public Administration)'], wacrac['RAC:Total Jobs'])
wacrac['RAC:NAICS%:92Public Administration'] = percent(wacrac['RAC:NAICS:92Public Administration'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Race%:White Alone'] = percent(wacrac['RAC:Race:White Alone'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Race%:Black or African American Alone'] = percent(wacrac['RAC:Race:Black or African American Alone'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Race%:American Indian or Alaska Native Alone'] = percent(wacrac['RAC:Race:American Indian or Alaska Native Alone'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Race%:Asian Alone'] = percent(wacrac['RAC:Race:Asian Alone'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Race%:Native Hawaiian or Other Pacific Islander Alone'] = percent(wacrac['RAC:Race:Native Hawaiian or Other Pacific Islander Alone'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Race%:Two or More Races'] = percent(wacrac['RAC:Race:Two or More Races'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Ethnicity%:Not Hispanic or Latino'] = percent(wacrac['RAC:Ethnicity:Not Hispanic or Latino'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Ethnicity%:Hispanic or Latino'] = percent(wacrac['RAC:Ethnicity:Hispanic or Latino'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Ed%:Less than High School'] = percent(wacrac['RAC:Ed:Less than High School'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Ed%:High School or Equivalent'] = percent(wacrac['RAC:Ed:High School or Equivalent'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Ed%:Some College or Associates'] = percent(wacrac['RAC:Ed:Some College or Associates'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Ed%:Bachelors or Advanced Degree'] = percent(wacrac['RAC:Ed:Bachelors or Advanced Degree'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Sex%:Male'] = percent(wacrac['RAC:Sex:Male'], wacrac['RAC:Total Jobs'])
wacrac['RAC:Sex%:Female'] = percent(wacrac['RAC:Sex:Female'], wacrac['RAC:Total Jobs'])

In [155]:
#make sure numerical data is float datatype
cols = list(wacrac.columns)
#cols.remove('NAME')
wacrac[cols] = wacrac[cols].astype(float)

In [157]:
wacrac.reset_index(drop = False, inplace = True)

In [158]:
wacrac['GEO_ID'] = wacrac['NAME'].map(geotogeoid)
#EXTRA INDEX WHY
wacrac['Source'] = 'LEHD'

In [159]:
#final check
wacrac.head()

,NAME,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation & Warehousing,RAC:NAICS:51Information,RAC:NAICS:52Finance & Insurance,RAC:NAICS:53Real Estate & Rental & Leasing,"RAC:NAICS:54Professional, Scientific, & Technical Services",RAC:NAICS:55Management of Companies & Enterprises,RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS:61Educational Services,RAC:NAICS:62Healthcare & Social Assistance,"RAC:NAICS:71Arts, Entertainment, & Recreation",RAC:NAICS:72Accommodation & Food Services,RAC:NAICS:81Other Services (Except Public Administration),RAC:NAICS:92Public Administration,RAC:Race:White Alone,RAC:Race:Black or African American Alone,RAC:Race:American Indian or Alaska Native Alone,RAC:Race:Asian Alone,RAC:Race:Native Hawaiian or Other Pacific Islander Alone,RAC:Race:Two or More Races,RAC:Ethnicity:Not Hispanic or Latino,RAC:Ethnicity:Hispanic or Latino,RAC:Ed:Less than High School,RAC:Ed:High School or Equivalent,RAC:Ed:Some College or Associates,RAC:Ed:Bachelors or Advanced Degree,RAC:Sex:Male,RAC:Sex:Female,WAC:Age%:Workers 29 or Younger,WAC:Age%:Workers 30 to 54,WAC:Age%:Workers 55 Older,WAC:Earnings%:1250mo or Less,WAC:Earnings%:1251 to 3333mo,WAC:Earnings%:Greater Than 3333mo,"WAC:NAICS%:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS%:21Mining, Quarrying, Oil & Gas",WAC:NAICS%:22Utilities,WAC:NAICS%:23Construction,WAC:NAICS%:31to33Manufacturing,WAC:NAICS%:42Wholesale Trade,WAC:NAICS%:44to45Retail Trade,WAC:NAICS%:48to49Transportation & Warehousing,WAC:NAICS%:51Information,WAC:NAICS%:52Finance & Insurance,WAC:NAICS%:53Real Estate & Rental & Leasing,"WAC:NAICS%:54Professional, Scientific, & Technical Services",WAC:NAICS%:55Management of Companies & Enterprises,WAC:NAICS%:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS%:61Educational Services,WAC:N

In [160]:
wacrac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Columns: 185 entries, NAME to Source
dtypes: float64(182), object(3)
memory usage: 117.2+ KB


In [161]:
conn = sq.connect('../../Outputs/Labor_Economy.db')
wacrac.to_sql('LEHD_WacRac_ACS5_2019', conn, if_exists = 'replace', index = False)

81

In [162]:
ruth = wacrac.loc[wacrac['NAME'] == 'Rutherford County']
ruth

,NAME,WAC:Total Jobs,WAC:Age:Workers 29 or Younger,WAC:Age:Workers 30 to 54,WAC:Age:Workers 55 Older,WAC:Earnings:1250mo or Less,WAC:Earnings:1251 to 3333mo,WAC:Earnings:Greater Than 3333mo,"WAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS:21Mining, Quarrying, Oil & Gas",WAC:NAICS:22Utilities,WAC:NAICS:23Construction,WAC:NAICS:31to33Manufacturing,WAC:NAICS:42Wholesale Trade,WAC:NAICS:44to45Retail Trade,WAC:NAICS:48to49Transportation & Warehousing,WAC:NAICS:51Information,WAC:NAICS:52Finance & Insurance,WAC:NAICS:53Real Estate & Rental & Leasing,"WAC:NAICS:54Professional, Scientific, & Technical Services",WAC:NAICS:55Management of Companies & Enterprises,WAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS:61Educational Services,WAC:NAICS:62Healthcare & Social Assistance,"WAC:NAICS:71Arts, Entertainment, & Recreation",WAC:NAICS:72Accommodation & Food Services,WAC:NAICS:81Other Services (Except Public Administration),WAC:NAICS:92Public Administration,WAC:Race:White Alone,WAC:Race:Black or African American Alone,WAC:Race:American Indian or Alaska Native Alone,WAC:Race:Asian Alone,WAC:Race:Native Hawaiian or Other Pacific Islander Alone,WAC:Race:Two or More Races,WAC:Ethnicity:Not Hispanic or Latino,WAC:Ethnicity:Hispanic or Latino,WAC:Ed:Less than High School,WAC:Ed:High School or Equivalent,WAC:Ed:Some College or Associates,WAC:Ed:Bachelors or Advanced Degree,WAC:Sex:Male,WAC:Sex:Female,WAC:FirmAge:0 to 1 Year,WAC:FirmAge:2 to 3 Years,WAC:FirmAge:4 to 5 Years,WAC:FirmAge:6 to 10 Years,WAC:FirmAge:11 or More Years,WAC:FirmSize:0 to 19 Employees,WAC:FirmSize:20 to 49 Employees,WAC:FirmSize:50 to 249 Employees,WAC:FirmSize:250 to 499 Employees,WAC:FirmSize:500 or More Employees,RAC:Total Jobs,RAC:Age:Workers 29 or Younger,RAC:Age:Workers 30 to 54,RAC:Age:Workers 55 Older,RAC:Earnings:1250mo or Less,RAC:Earnings:1251 to 3333mo,RAC:Earnings:Greater Than 3333mo,"RAC:NAICS:11Agriculture, Forestry, Fishing & Hunting","RAC:NAICS:21Mining, Quarrying, Oil & Gas",RAC:NAICS:22Utilities,RAC:NAICS:23Construction,RAC:NAICS:31to33Manufacturing,RAC:NAICS:42Wholesale Trade,RAC:NAICS:44to45Retail Trade,RAC:NAICS:48to49Transportation & Warehousing,RAC:NAICS:51Information,RAC:NAICS:52Finance & Insurance,RAC:NAICS:53Real Estate & Rental & Leasing,"RAC:NAICS:54Professional, Scientific, & Technical Services",RAC:NAICS:55Management of Companies & Enterprises,RAC:NAICS:56Administrative & Support & Waste Management & Remediation Services,RAC:NAICS:61Educational Services,RAC:NAICS:62Healthcare & Social Assistance,"RAC:NAICS:71Arts, Entertainment, & Recreation",RAC:NAICS:72Accommodation & Food Services,RAC:NAICS:81Other Services (Except Public Administration),RAC:NAICS:92Public Administration,RAC:Race:White Alone,RAC:Race:Black or African American Alone,RAC:Race:American Indian or Alaska Native Alone,RAC:Race:Asian Alone,RAC:Race:Native Hawaiian or Other Pacific Islander Alone,RAC:Race:Two or More Races,RAC:Ethnicity:Not Hispanic or Latino,RAC:Ethnicity:Hispanic or Latino,RAC:Ed:Less than High School,RAC:Ed:High School or Equivalent,RAC:Ed:Some College or Associates,RAC:Ed:Bachelors or Advanced Degree,RAC:Sex:Male,RAC:Sex:Female,WAC:Age%:Workers 29 or Younger,WAC:Age%:Workers 30 to 54,WAC:Age%:Workers 55 Older,WAC:Earnings%:1250mo or Less,WAC:Earnings%:1251 to 3333mo,WAC:Earnings%:Greater Than 3333mo,"WAC:NAICS%:11Agriculture, Forestry, Fishing & Hunting","WAC:NAICS%:21Mining, Quarrying, Oil & Gas",WAC:NAICS%:22Utilities,WAC:NAICS%:23Construction,WAC:NAICS%:31to33Manufacturing,WAC:NAICS%:42Wholesale Trade,WAC:NAICS%:44to45Retail Trade,WAC:NAICS%:48to49Transportation & Warehousing,WAC:NAICS%:51Information,WAC:NAICS%:52Finance & Insurance,WAC:NAICS%:53Real Estate & Rental & Leasing,"WAC:NAICS%:54Professional, Scientific, & Technical Services",WAC:NAICS%:55Management of Companies & Enterprises,WAC:NAICS%:56Administrative & Support & Waste Management & Remediation Services,WAC:NAICS%:61Educational Services,WAC:N

## Origin-Destination

In [163]:
#all primary jobs
url = 'https://lehd.ces.census.gov/data/lodes/LODES7/tn/od/tn_od_main_JT01_2019.csv.gz'
od = pd.read_csv(url)
print('Your file is ready.')

Your file is ready.


In [164]:
odcols = {'h_geocode': 'HOMEGEO_ID', 'w_geocode': 'WORKGEO_ID', 'S000': 'Total Jobs', 'SA01': 'Age:Workers 29 or Younger', 'SA02': 'Age:Workers 30 to 54', 
          'SA03': 'Age:Workers 55 Older', 'SE01': 'Earnings:1250mo or Less', 'SE02': 'Earnings:1251 to 3333mo', 'SE03': 'Earnings:Greater Than 3333mo', 
          'SI01': 'Sector:Goods Producing', 'SI02': 'Sector:Trade, Transportation & Utilities', 'SI03': 'Sector:All Other Services'}

In [165]:
od.rename(columns = odcols, inplace = True)

In [166]:
od.head(2)

,WORKGEO_ID,HOMEGEO_ID,Total Jobs,Age:Workers 29 or Younger,Age:Workers 30 to 54,Age:Workers 55 Older,Earnings:1250mo or Less,Earnings:1251 to 3333mo,Earnings:Greater Than 3333mo,Sector:Goods Producing,"Sector:Trade, Transportation & Utilities",Sector:All Other Services,createdate
0,470010201001000,470010201001063,2,1,1,0,0,1,1,0,0,2,20211018
1,470010201001000,470010201002013,1,0,0,1,1,0,0,0,0,1,20211018


In [167]:
od['WORKGEO_ID'] = od['WORKGEO_ID'].astype(str)
od['HOMEGEO_ID'] = od['HOMEGEO_ID'].astype(str)

In [168]:
#add columns with county name and place name respectively
od['Home County'] = od['HOMEGEO_ID'].map(c_inverted)
od['Home Incorporated'] = od['HOMEGEO_ID'].map(un_inverted)
od['Home Place'] = od['HOMEGEO_ID'].map(p_inverted)

od['Work County'] = od['WORKGEO_ID'].map(c_inverted)
od['Work Incorporated'] = od['WORKGEO_ID'].map(un_inverted)
od['Work Place'] = od['WORKGEO_ID'].map(p_inverted)

In [170]:
od['Home County'] = od['Home County'].str.get(0)
od['Home Incorporated'] = od['Home Incorporated'].str.get(0)
od['Home Place'] = od['Home Place'].str.get(0)

od['Work County'] = od['Work County'].str.get(0)
od['Work Incorporated'] = od['Work Incorporated'].str.get(0)
od['Work Place'] = od['Work Place'].str.get(0)

In [172]:
test = od['Home County'].str.split(pat = ",", expand = True)
od['Home County'] = test[0]
test = od['Work County'].str.split(pat = ",", expand = True)
od['Work County'] = test[0]

In [173]:
test = od['Home Place'].str.split(pat = ",", expand = True)
od['Home Place'] = test[0]
test = od['Work Place'].str.split(pat = ",", expand = True)
od['Work Place'] = test[0]

In [174]:
od = od.drop(columns = 'createdate')

In [175]:
od.head()

,WORKGEO_ID,HOMEGEO_ID,Total Jobs,Age:Workers 29 or Younger,Age:Workers 30 to 54,Age:Workers 55 Older,Earnings:1250mo or Less,Earnings:1251 to 3333mo,Earnings:Greater Than 3333mo,Sector:Goods Producing,"Sector:Trade, Transportation & Utilities",Sector:All Other Services,Home County,Home Incorporated,Home Place,Work County,Work Incorporated,Work Place
0,470010201001000,470010201001063,2,1,1,0,0,1,1,0,0,2,Anderson County,NaN,Oak Ridge city,Anderson County,NaN,Oak Ridge city
1,470010201001000,470010201002013,1,0,0,1,1,0,0,0,0,1,Anderson County,NaN,Oak Ridge city,Anderson County,NaN,Oak Ridge city
2,470010201001000,470010201002017,1,1,0,0,0,1,0,0,0,1,Anderson County,NaN,Oak Ridge city,Anderson County,NaN,Oak Ridge city
3,470010201001000,470010201002021,1,1,0,0,0,1,0,0,0,1,Anderson County,NaN,Oak Ridge city,Anderson County,NaN,Oak Ridge city
4,470010201001000,470010202023018,1,0,0,1,0,0,1,0,0,1,Anderson County,NaN,Oak Ridge city,Anderson County,NaN,Oak Ridge city


In [176]:
od['Work Incorporated'].unique()

array([nan, 'None', 'Andersonville CDP, TN', 'Unionville CDP, TN',
       'Eagleton Village CDP, TN', 'Seymour CDP, TN', 'Wildwood CDP, TN',
       'Walland CDP, TN', 'East Cleveland CDP, TN',
       'Wildwood Lake CDP, TN', 'South Cleveland CDP, TN',
       'Hopewell CDP, TN', 'Fincastle CDP, TN', 'Pine Crest CDP, TN',
       'Central CDP, TN', 'Hunter CDP, TN', 'Roan Mountain CDP, TN',
       'New Union CDP, TN', 'Lakewood Park CDP, TN', 'Hillsboro CDP, TN',
       'Fairfield Glade CDP, TN', 'Bowman CDP, TN', 'Lake Tansi CDP, TN',
       'Grimsley CDP, TN', 'Clarkrange CDP, TN', 'Sewanee CDP, TN',
       'Fall Branch CDP, TN', 'Pelham CDP, TN', 'Sale Creek CDP, TN',
       'Mowbray Mountain CDP, TN', 'Middle Valley CDP, TN',
       'Falling Water CDP, TN', 'Fairmount CDP, TN', 'Ooltewah CDP, TN',
       'Apison CDP, TN', 'Harrison CDP, TN', 'Olivet CDP, TN',
       'Walnut Grove CDP, TN', 'Mooresburg CDP, TN', 'Darden CDP, TN',
       'Chesterfield CDP, TN', 'Bon Aqua Junction CDP, T

In [177]:
od['Home County'] = od['Home County'] + ', Tennessee'
od['Home Place'] = od['Home Place'] + ', Tennessee'
od['Work County'] = od['Work County'] + ', Tennessee'
od['Work Place'] = od['Work Place'] + ', Tennessee'

In [178]:
od['Source'] = 'LEHD'

In [179]:
#final check
od.head()

,WORKGEO_ID,HOMEGEO_ID,Total Jobs,Age:Workers 29 or Younger,Age:Workers 30 to 54,Age:Workers 55 Older,Earnings:1250mo or Less,Earnings:1251 to 3333mo,Earnings:Greater Than 3333mo,Sector:Goods Producing,"Sector:Trade, Transportation & Utilities",Sector:All Other Services,Home County,Home Incorporated,Home Place,Work County,Work Incorporated,Work Place,Source
0,470010201001000,470010201001063,2,1,1,0,0,1,1,0,0,2,"Anderson County, Tennessee",NaN,"Oak Ridge city, Tennessee","Anderson County, Tennessee",NaN,"Oak Ridge city, Tennessee",LEHD
1,470010201001000,470010201002013,1,0,0,1,1,0,0,0,0,1,"Anderson County, Tennessee",NaN,"Oak Ridge city, Tennessee","Anderson County, Tennessee",NaN,"Oak Ridge city, Tennessee",LEHD
2,470010201001000,470010201002017,1,1,0,0,0,1,0,0,0,1,"Anderson County, Tennessee",NaN,"Oak Ridge city, Tennessee","Anderson County, Tennessee",NaN,"Oak Ridge city, Tennessee",LEHD
3,470010201001000,470010201002021,1,1,0,0,0,1,0,0,0,1,"Anderson County, Tennessee",NaN,"Oak Ridge city, Tennessee","Anderson County, Tennessee",NaN,"Oak Ridge city, Tennessee",LEHD
4,470010201001000,470010202023018,1,0,0,1,0,0,1,0,0,1,"Anderson County, Tennessee",NaN,"Oak Ridge city, Tennessee","Anderson County, Tennessee",NaN,"Oak Ridge city, Tennessee",LEHD


In [180]:
od.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2273811 entries, 0 to 2273810
Data columns (total 19 columns):
 #   Column                                    Dtype 
---  ------                                    ----- 
 0   WORKGEO_ID                                object
 1   HOMEGEO_ID                                object
 2   Total Jobs                                int64 
 3   Age:Workers 29 or Younger                 int64 
 4   Age:Workers 30 to 54                      int64 
 5   Age:Workers 55 Older                      int64 
 6   Earnings:1250mo or Less                   int64 
 7   Earnings:1251 to 3333mo                   int64 
 8   Earnings:Greater Than 3333mo              int64 
 9   Sector:Goods Producing                    int64 
 10  Sector:Trade, Transportation & Utilities  int64 
 11  Sector:All Other Services                 int64 
 12  Home County                               object
 13  Home Incorporated                         object
 14  Home Place        

In [181]:
conn = sq.connect('../../Outputs/Labor_Economy.db')
od.to_sql('LEHD_OD_ACS5_2019', conn, if_exists = 'replace', index = False)

2273811